In [53]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
# import cv2
# import csv
import os
import sys
import time
import struct
import h5py
import scipy.io as sio
# from scipy import ndimage
from numpy import linalg as LA
from IPython.display import display, Image
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import tensorflow as tf

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [70]:
import scipy.io
# Load synthetic dataset

X = scipy.io.loadmat('/Users/angelsrates/Documents/PhD/4th Semester/Project/poles_data2.mat')
y = scipy.io.loadmat('/Users/angelsrates/Documents/PhD/4th Semester/Project/poles_y2.mat')
data = X['data']
data = np.squeeze(np.transpose(data))
#data_noise = X['data_noise']
#data_noise = np.squeeze(np.transpose(data_noise))
#sys_par = np.squeeze(np.transpose(X['dic_par']))
#sys_par = [np.append(np.array([-1]), sys_par[i]) for i in range(sys_par.shape[0])]
y = y['label']
y = np.squeeze(y - 1)
n_classes = max(y) + 1
#num_poles = np.squeeze(X['num_poles'])
num_poles = 2

In [71]:
sys_par = [[-1,1.39954943237774, -1], [-1,0.411382829503097, -1]]

In [72]:
np.random.seed(4294967295)
[N, T] = data.shape
permutation = np.random.permutation(data.shape[0])
data = [data[perm] for perm in permutation]
y = [y[perm] for perm in permutation]
X = data
# If data with noise, change to:
# X = data_noise

In [73]:
#Select training and testing (75% and 25%)
thr = int(N*0.75)
y = [int(i) for i in y]
X_train = np.asarray(X[:thr])
y_train = np.asarray(y[:thr])
X_test = np.asarray(X[thr:])
y_test = np.asarray(y[thr:])

print('Training data size', X_train.shape)
print('Training Ground-Truth size', y_train.shape)
print('Testing data size', X_test.shape)
print('Testing Ground-Truth size', y_test.shape)

Training data size (225, 50)
Training Ground-Truth size (225,)
Testing data size (75, 50)
Testing Ground-Truth size (75,)


In [74]:
def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data. 
    
    shape = list(_train.shape)
    #shape = list((batch_size, 1843200))
    shape[0] = batch_size
    #shape[1] = 1843200
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index]
        #batch_s[i] = np.reshape(load_video(_train[index]), (1,1843200))

    return batch_s

def one_hot(y_):
    # Function to encode output labels from number indexes 
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    
    y_ = y_.reshape(len(y_))
    n_values = np.max(y_) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

In [75]:
from scipy import signal
import control
from scipy.signal import step2
import math

# Parameters
learning_rate = 0.0015
batch_size = 1

# Network Parameters
n_input = T
#dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float64, [n_input])
y = tf.placeholder(tf.float32, [1, n_classes])
#labels = tf.placeholder(tf.int32, [1,1])

def index_along_every_row(array, index):
    N,_ = array.shape 
    return array[np.arange(N), index]

def build_hankel_tensor(x, nr, nc, N, dim):
    cidx = np.arange(0, nc, 1)
    ridx = np.transpose(np.arange(1, nr+1, 1))
    Hidx = np.transpose(np.tile(ridx, (nc,1))) + dim*np.tile(cidx, (nr,1))
    Hidx = Hidx - 1
    arr = tf.reshape(x[:], (1,N))
    return tf.py_func(index_along_every_row, [arr, Hidx], [tf.float64])[0]

def build_hankel(x, nr, nc, N, dim):
    cidx = np.arange(0, nc, 1)
    ridx = np.transpose(np.arange(1, nr+1, 1))
    Hidx = np.transpose(np.tile(ridx, (nc,1))) + dim*np.tile(cidx, (nr,1))
    Hidx = Hidx - 1
    arr = x[:]
    return arr[Hidx]

# Create model
def poles_net(x, sys_par, T, num_poles):
    # Operate over single-channel trajectories
    # Sampling rates at 0.3
    W_col = []
    for i in range(num_poles):
        sys = control.TransferFunction([1, 0], sys_par[i], 0.3)
        [y1, _] = control.matlab.impulse(sys, T=np.arange(T))
        y1 = tf.transpose(y1[0,:T])
        W_col.append(y1)
    W = tf.reshape(tf.stack(W_col, axis=1), (T,num_poles))
    coeff = tf.abs(tf.matrix_solve_ls(W, tf.reshape(x, (T,1)), l2_regularizer=0.0, fast=False, name=None))
    coeff = tf.transpose(coeff)
    out = tf.add(tf.matmul(tf.cast(coeff, tf.float32), weights['out']), biases['out'])
    return [coeff, out]

weights = {
    'out': tf.Variable(tf.random_normal([num_poles, n_classes]))
}
    
biases = {
    'out': tf.Variable(tf.random_normal([1, n_classes]))
}

[coeff, pred]= poles_net(x, sys_par, T, num_poles)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

In [76]:
y_test = one_hot(y_test)

In [77]:
# Launch the graph
n_epochs = 1
training_iters = X_train.shape[0]*n_epochs
display_step = 1
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    train_acc = 0
    while step * batch_size <= training_iters:
        batch_x = np.squeeze(extract_batch_size(X_train,step,batch_size))
        batch_y = extract_batch_size(one_hot(y_train),step,batch_size)
        #batch_y = np.reshape(extract_batch_size(y_train,step,batch_size), (1,1))
        print(batch_y.shape)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})
            train_acc += acc 
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print('Final Training Accuracy:', train_acc/(X_train.shape[0]*n_epochs))
    print("Optimization Finished!")
    acc = 0
    for i in range(X_test.shape[0]):
        test = np.squeeze(X_test[i,:])
        label = np.reshape(y_test[i,:], (1,n_classes))
        #label = np.reshape(y_test[i], (1,1))
        print(label)
        print("Trajectory:", i, \
            sess.run([coeff], feed_dict={x: test, y: label}))
        print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={x: test, y: label}))
        acc += sess.run(accuracy, feed_dict={x: test, y: label})
    print('Final Testing Accuracy:', acc/X_test.shape[0])

(1, 3)
Iter 1, Minibatch Loss= 0.463941, Training Accuracy= 1.00000
(1, 3)
Iter 2, Minibatch Loss= 1.239681, Training Accuracy= 0.00000
(1, 3)
Iter 3, Minibatch Loss= 0.603769, Training Accuracy= 1.00000
(1, 3)
Iter 4, Minibatch Loss= 0.347242, Training Accuracy= 1.00000
(1, 3)
Iter 5, Minibatch Loss= 0.234731, Training Accuracy= 1.00000
(1, 3)
Iter 6, Minibatch Loss= 0.961255, Training Accuracy= 0.00000
(1, 3)
Iter 7, Minibatch Loss= 0.455640, Training Accuracy= 1.00000
(1, 3)
Iter 8, Minibatch Loss= 0.563217, Training Accuracy= 1.00000
(1, 3)
Iter 9, Minibatch Loss= 1.285264, Training Accuracy= 0.00000
(1, 3)
Iter 10, Minibatch Loss= 0.582287, Training Accuracy= 1.00000
(1, 3)
Iter 11, Minibatch Loss= 0.341521, Training Accuracy= 1.00000
(1, 3)
Iter 12, Minibatch Loss= 0.809583, Training Accuracy= 1.00000
(1, 3)
Iter 13, Minibatch Loss= 1.004590, Training Accuracy= 1.00000
(1, 3)
Iter 14, Minibatch Loss= 0.583472, Training Accuracy= 1.00000
(1, 3)
Iter 15, Minibatch Loss= 0.725431, T